In [40]:
import os
import sys

import numpy as np
import pandas as pd
import tensorflow as tf

# BERT
bert_path = "C:/Users/roksi/bert"

if not bert_path in sys.path:
    sys.path.append(bert_path)

import modeling
import tokenization


data_path = "./data/"

data_val = pd.read_csv(data_path + "cloze_test_val__spring2016 - cloze_test_ALL_val.csv")
data_train = pd.read_csv(data_path + "train_stories.csv")

In [132]:
def pp_text_a(row):
    return row[1] + " " + row[2] + " " + row[3] + " " + row[4]
    
def pp_text_b_pos(row):
    if row[7] == 1:
        return row[5]
    elif row[7] == 2:
        return row[6]
    
def pp_text_b_neg(row):
    if row[7] == 1:
        return row[6]
    elif row[7] == 2:
        return row[5]

In [134]:
df_pos = pd.DataFrame()
df_pos["guid"] = data_val["InputStoryid"]
df_pos["text_a"] = data_val.apply(pp_text_a, axis=1)
df_pos["text_b"] = data_val.apply(pp_text_b_pos, axis=1)
df_pos["label"] = 1 # 1 -> positive/correct class

df_neg = pd.DataFrame()
df_neg["guid"] = data_val["InputStoryid"]
df_neg["text_a"] = data_val.apply(pp_text_a, axis=1)
df_neg["text_b"] = data_val.apply(pp_text_b_neg, axis=1)
df_neg["label"] = 0

df = pd.concat([df_pos, df_neg], axis=0).sample(frac=1).reset_index(drop=True)
df.head(3)
# df.apply(lambda row: len(row[1]), axis=1).max()

,guid,text_a,text_b,label
0,449c1058-fb27-4c6d-9cac-fbd5b32b80da,Billy buried some treasure in his backyard. He...,He threw away the treasure he found.,0
1,112151f0-6986-4251-aed3-2c454670a09f,Ellen used to live in the country. She misses ...,Ellen was perfectly comfortable.,0
2,112151f0-6986-4251-aed3-2c454670a09f,Ellen used to live in the country. She misses ...,She was happy when the power came back on.,1


In [108]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = bert_path + "/bert/"

MAX_SEQ_LENGTH = 128
BERT_CONFIG = BERT_PRETRAINED_DIR + '/bert_config.json'
CHKPT_DIR = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
VOCAB_FILE = BERT_PRETRAINED_DIR + '/vocab.txt'
INIT_CHECKPOINT = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
BATCH_SIZE = 128


print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))

***** BERT pretrained directory: C:/Users/roksi/bert/bert/ *****


In [140]:
from run_classifier import InputExample
from run_classifier import convert_examples_to_features
from run_classifier import input_fn_builder

MAX_SEQ_LENGTH = 256
label_list = [0, 1]

tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=True)

df_InputExamples = df.apply(lambda x: InputExample(guid=x[0], text_a=x[1], text_b=x[2], label=x[3]), axis=1)

df_features = convert_examples_to_features(df_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 12
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: 449c1058-fb27-4c6d-9cac-fbd5b32b80da
INFO:tensorflow:tokens: [CLS] billy buried some treasure in his backyard . he drew a map so he wouldn ' t forget where he ' d put it . years passed and billy ' s family moved . decades later , he went back and used his map to dig it back up . [SEP] he threw away the treasure he found . [SEP]
INFO:tensorflow:input_ids: 101 5006 3950 2070 8813 1999 2010 16125 1012 2002 3881 1037 4949 2061 2002 2876 1005 1056 5293 2073 2002 1005 1040 2404 2009 1012 2086 2979 1998 5006 1005 1055 2155 2333 1012 5109 2101 1010 2002 2253 2067 1998 2109 2010 4949 2000 10667 2009 2067 2039 1012 102 2002 4711 2185 1996 8813 2002 2179 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

INFO:tensorflow:label: 1 (id = 1)
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: c2699d68-4f16-44d1-bff6-6a66ec266a46
INFO:tensorflow:tokens: [CLS] mike had a free saturday . he drove to several car dealers . he was interested in seeing potential future cars for himself . mike found a jeep he fell in love with . [SEP] mike never liked jeep ##s . [SEP]
INFO:tensorflow:input_ids: 101 3505 2018 1037 2489 5095 1012 2002 5225 2000 2195 2482 16743 1012 2002 2001 4699 1999 3773 4022 2925 3765 2005 2370 1012 3505 2179 1037 14007 2002 3062 1999 2293 2007 1012 102 3505 2196 4669 14007 2015 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [156]:
# from extract_features import read_examples

tf.reset_default_graph()

tf.logging.set_verbosity(tf.logging.INFO)

layer_indexes = [-1, -2]

bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=True)

# examples = read_examples("tmp_input.txt")





In [141]:
BERT_CONFIG

'C:/Users/roksi/bert/bert//bert_config.json'

In [147]:
df

,guid,text_a,text_b,label
0,449c1058-fb27-4c6d-9cac-fbd5b32b80da,Billy buried some treasure in his backyard. He...,He threw away the treasure he found.,0
1,112151f0-6986-4251-aed3-2c454670a09f,Ellen used to live in the country. She misses ...,Ellen was perfectly comfortable.,0
2,112151f0-6986-4251-aed3-2c454670a09f,Ellen used to live in the country. She misses ...,She was happy when the power came back on.,1
3,18d4dbc1-ed3c-40ca-8155-18884bb42732,I went to buy a new television last week. Afte...,I returned the TV.,1
4,c2699d68-4f16-44d1-bff6-6a66ec266a46,Mike had a free Saturday. He drove to several ...,Mike never liked Jeeps.,0
5,0e959d20-1b35-4853-9e5b-6e5e6a015d32,Amy was getting a minor raise from her job. Al...,Amy was relieved she didn't get any extra money.,0
6,c2699d68-4f16-44d1-bff6-6a66ec266a46,Mike had a free Saturday. He drove to several ...,Mike bought the Jeep.,1
7,449c1058-fb27-4c6d-9cac-fbd5b32b80da,Billy buried some treasure in his backyard. He...,He found many treasures from his childhood.,1
8,18d4dbc1-ed3c-40ca-8155-18884bb42732,I went to buy a new television last week. Afte...,I was happy with the TV.,0
9,11871d4b-6b94-4184-b36d-c4e76d311f3d,Peter was excited to go to the Sanders rally i...,He couldn't wait to vote for him.,1


In [164]:
# with open("sample_input.txt", "w") as f:
    for _, row in df.iterrows():
        line = row["text_b"]
        f.write(line + "\n")
        
    

Billy buried some treasure in his backyard. He drew a map so he wouldn't forget where he'd put it. Years passed and Billy's family moved. Decades later, he went back and used his map to dig it back up. He threw away the treasure he found.
Ellen used to live in the country. She misses having days with the power out. One day an ice storm caused the power to go out in her city home. She got very cold. Ellen was perfectly comfortable.
Ellen used to live in the country. She misses having days with the power out. One day an ice storm caused the power to go out in her city home. She got very cold. She was happy when the power came back on.
I went to buy a new television last week. After finally picking one out I took it home. I noticed it was broken as soon as I got it out of the box. The screen was cracked. I returned the TV.
Mike had a free Saturday. He drove to several car dealers. He was interested in seeing potential future cars for himself. Mike found a Jeep he fell in love with. Mike n

In [160]:
features = df_features

unique_id_to_feature = {}
for feature in features:
    unique_id_to_feature[feature.guid] = feature


AttributeError: 'InputFeatures' object has no attribute 'guid'

In [163]:
features[0].is_real_example

True

In [ ]:
from extract_features import model_fn_builder
model_fn = model_fn_builder()